In [58]:
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
train_df=pd.read_csv('Disaster_train.csv')
test_df=pd.read_csv('Disaster_test.csv')


In [61]:
train_df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [62]:
train_df.isnull().sum()

,0
id,0
keyword,61
location,2533
text,0
target,0


In [63]:
train_df['keyword']=train_df['keyword'].fillna('None')
train_df['location']=train_df['location'].fillna('Unknown')

In [64]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [65]:
# Function to clean text
def clean_text(text):
    text= text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    return text

In [66]:
# Clean the text
train_df['cleaned_text'] = train_df['text'].apply(clean_text)

In [67]:
import nltk

# ... your existing code ...

# Download 'punkt_tab' before using word_tokenize
nltk.download('punkt_tab')

# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

# ... your existing code ...

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [68]:
# Function for stemming
from nltk.stem import PorterStemmer # Import PorterStemmer

def stem_text(text):
   stemmer = PorterStemmer() # Create a PorterStemmer instance
   words = word_tokenize(text)
   return ' '.join([stemmer.stem(word) for word in words])

train_df['cleaned_text'] = train_df['cleaned_text'].apply(stem_text)

In [69]:
# Function for lemmatization
from nltk.stem import WordNetLemmatizer  # Import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()  # Create a WordNetLemmatizer instance
    words = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in words])

# Apply lemmatization
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lemmatize_text)

In [70]:
# Tokenize the cleaned text for Word2Vec
tokenized_text = [word_tokenize(text) for text in train_df['cleaned_text']]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use
word2vec_model.save("word2vec.model")

In [71]:
# Function to get average Word2Vec vectors for each text
def get_vector(text):
    words = word_tokenize(text)
    vector = np.zeros(100)  # Assuming vector_size=100
    count = 0
    for word in words:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

# Apply the function to get vectors
train_df['text_vector'] = train_df['cleaned_text'].apply(get_vector)

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Prepare features and labels
X = np.array(train_df['text_vector'].tolist())
y = train_df['target']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7248850952068286


In [73]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# Train a model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6986211424819435


In [74]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler  # Import MinMaxScaler for scaling
from sklearn.pipeline import Pipeline # Import the Pipeline class

# ... your existing code ...

# Train a model with scaling
# Create a pipeline to first scale the data and then apply MultinomialNB
model = Pipeline([('scaler', MinMaxScaler()), ('clf', MultinomialNB())])
model.fit(X_train, y_train)  # Train the model

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.5600787918581747


In [75]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [76]:
test_df.isnull().sum()

,0
id,0
keyword,26
location,1105
text,0


In [77]:
test_df['keyword']=test_df['keyword'].fillna('None')
test_df['location']=test_df['location'].fillna('Unknown')

In [78]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [79]:
# Function to clean text
def clean_text(text):
    text= text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    return text

In [80]:
# Clean the text
test_df['cleaned_text'] = test_df['text'].apply(clean_text)

In [81]:
import nltk

# ... your existing code ...

# Download 'punkt_tab' before using word_tokenize
nltk.download('punkt_tab')

# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [82]:
# Function for stemming
def stem_text(text):
   words = word_tokenize(text)
   return ' '.join([stemmer.stem(word) for word in words])

test_df['cleaned_text'] = train_df['text'].apply(clean_text)

In [83]:
# Function for lemmatization
from nltk.stem import WordNetLemmatizer  # Import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()  # Create a WordNetLemmatizer instance here
    words = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in words])

# Apply lemmatization
test_df['cleaned_text'] = test_df['cleaned_text'].apply(lemmatize_text)


In [84]:
# Tokenize the cleaned text for Word2Vec
tokenized_text = [word_tokenize(text) for text in test_df['cleaned_text']]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use
word2vec_model.save("word2vec.model")

In [85]:
# Function to get average Word2Vec vectors for each text
def get_vector(text):
    words = word_tokenize(text)
    vector = np.zeros(100)  # Assuming vector_size=100
    count = 0
    for word in words:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

# Apply the function to get vectors
test_df['text_vector'] = test_df['cleaned_text'].apply(get_vector)

In [86]:
np.array(test_df['text_vector'].tolist()) # Changed train_df to test_df

array([[-0.22109298,  0.23662518,  0.13644184, ..., -0.20418291,
         0.22703167,  0.1896095 ],
       [-0.09275901,  0.10172625,  0.05298338, ..., -0.08738434,
         0.09803478,  0.07813324],
       [-0.18419324,  0.19910643,  0.11236725, ..., -0.17081122,
         0.19574685,  0.161579  ],
       ...,
       [-0.06705466,  0.07424149,  0.04033126, ..., -0.06157029,
         0.07040878,  0.05616103],
       [-0.22416538,  0.23756303,  0.13766057, ..., -0.20545466,
         0.23033377,  0.19183997],
       [-0.1907229 ,  0.20952893,  0.1219827 , ..., -0.18408553,
         0.20455017,  0.16997347]])

In [87]:
from sklearn.ensemble import RandomForestClassifier # Import the RandomForestClassifier

# ... your existing code ...

model = RandomForestClassifier()  # Instantiate the model

In [88]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [89]:
y_pred=model.predict(X_test)

In [90]:
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7235718975705844


In [91]:
test_df.head()

,id,keyword,location,text,cleaned_text,text_vector
0,0,None,Unknown,Just happened a terrible car crash,our deed are the reason of this earthquake may...,"[-0.22109298234303984, 0.23662518036480135, 0...."
1,2,None,Unknown,"Heard about #earthquake is different cities, s...",forest fire near la ronge sask canada,"[-0.09275901237768787, 0.10172624566725322, 0...."
2,3,None,Unknown,"there is a forest fire at spot pond, geese are...",all resident asked to shelter in place are bei...,"[-0.18419323871123858, 0.19910642533795908, 0...."
3,9,None,Unknown,Apocalypse lighting. #Spokane #wildfires,people receive wildfire evacuation order in ca...,"[-0.1609498467712131, 0.17050684017262288, 0.0..."
4,11,None,Unknown,Typhoon Soudelor kills 28 in China and Taiwan,just got sent this photo from ruby alaska a sm...,"[-0.2060532683681231, 0.21876102383248508, 0.1..."


In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
logistic=LogisticRegression()
logistic.fit(X_train,y_train)

LogisticRegression()

In [94]:
y_pred=logistic.predict(X_test)


In [95]:
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7235718975705844
